In [1]:
import os
import json
from tensorflow.keras.preprocessing.text import text_to_word_sequence,Tokenizer
from tensorflow.keras.layers import Dense , Embedding , LSTM , Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential , save_model,load_model

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Embedding, Dense , Dropout,LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
%matplotlib inline

In [2]:
os.makedirs('Data',exist_ok=True)


In [2]:
with open('Data/intent.json') as file:
    data = json.load(file)
## reading the json file data

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Hey',
    'Ekse',
    'Hi'],
   'responses': ['Hi stranger',
    'Yebo yes, how can I help',
    'Ey watsupp',
    'Ekse Hoezit',
    'Hola',
    'Hey dude',
    'You again']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'Ok bye', 'Bye Bye'],
   'responses': ['See you!',
    'Have a nice day',
    'Sure Bye',
    'Later dude',
    'Sayoonara',
    'Peace out',
    'Sure dude',
    'Ciao']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    'You are welcome',
    'Sure dude']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand',
    'I am still 

In [3]:
classes = len(data['intents'])
classes

8

In [5]:
classes

8

In [6]:
data['intents'][0]

{'tag': 'greeting',
 'patterns': ['Hi there',
  'How are you',
  'Is anyone there?',
  'Hey',
  'Hola',
  'Hello',
  'Good day',
  'Hey',
  'Ekse',
  'Hi'],
 'responses': ['Hi stranger',
  'Yebo yes, how can I help',
  'Ey watsupp',
  'Ekse Hoezit',
  'Hola',
  'Hey dude',
  'You again']}

In [3]:
tag = []
labels = []
pattern = []
response = []
for intent in data['intents']:
    label = intent['tag']
    response.append(intent['responses'])
    tag.append(label)
    for key , value in intent.items():
        if key == 'patterns':
            for patter in intent[key]: 
                pattern.append(patter)
                labels.append(label)
   
print(tag)
print()
print(labels)
print()
print(pattern)
print()
print(response,'\n')

print(f" len of response :- {len(response)}")

# successfully extracted the tag and labels with sentence

['greeting', 'goodbye', 'thanks', 'noanswer', 'name', 'options', 'south_africa_info', 'south_africa_facts']

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'name', 'name', 'name', 'name', 'name', 'name', 'name', 'options', 'options', 'options', 'options', 'options', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_info', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts', 'south_africa_facts']

['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Hey', 'Ekse', 'Hi'

In [7]:
vocabulary_size = 10000

In [12]:
import tensorflow_hub as hub
import tensorflow_text as text

NotFoundError: c:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_text\python\ops\_regex_split_ops.so not found

In [ ]:
!pip install tensorflow_text

In [7]:
process_model = hub.KerasLayer(r"https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

FileNotFoundError: Op type not registered 'CaseFoldUTF8' in binary running on DESKTOP-0VUKMTH. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [8]:
one_hoted = [one_hot(sentence,vocabulary_size) for sentence in pattern]
one_hoted

[[2263, 479],
 [9686, 6707, 7589],
 [7879, 3640, 479],
 [5073],
 [3505],
 [6446],
 [2592, 1506],
 [5073],
 [808],
 [2263],
 [1245],
 [637, 7589, 7547],
 [5698],
 [5388, 1245],
 [1245, 1245],
 [4767],
 [5915, 7589],
 [8552, 4512],
 [9630, 4767],
 [4767, 539, 3288, 1656],
 [4341, 7879, 9273, 9716],
 [9556, 9273, 9716],
 [8221, 1656, 9273, 9716],
 [4227, 6707, 7589],
 [4891],
 [8221, 1656, 4058, 7108],
 [7589, 6707],
 [4341, 7914, 7589, 7914],
 [9686, 1897, 7589, 2392],
 [4341, 7914, 7589, 1639],
 [4341, 7879, 9273, 1594],
 [9686, 1897, 7589, 2392],
 [3044, 7648],
 [4341, 1897, 7589, 8221, 1656, 4058, 1691],
 [1691],
 [8221, 1656, 4058, 1691],
 [4341, 4058, 1691],
 [4341, 7914, 7589, 1639, 4058, 1691],
 [8221, 1656, 4895, 4058, 1691],
 [4341, 1897, 7589, 8221, 1656, 4058, 3044, 7648],
 [8221, 1656, 4058, 3044, 7648],
 [4341, 4058, 3044, 7648],
 [4341, 7914, 7589, 1639, 4058, 3044, 7648],
 [8221, 1656, 4895, 4058, 3044, 7648],
 [3044, 7648, 1564],
 [1691, 1564],
 [5837, 1656, 5664, 1564, 4

In [11]:
maximum_length_of_sentence = max(map(len,pattern))
maximum_length_of_sentence

39

In [12]:
train_data  = pad_sequences(one_hoted,padding='pre',maxlen=maximum_length_of_sentence)
train_data

array([[   0,    0,    0, ...,    0, 2263,  479],
       [   0,    0,    0, ..., 9686, 6707, 7589],
       [   0,    0,    0, ..., 7879, 3640,  479],
       ...,
       [   0,    0,    0, ..., 3044, 7648, 1564],
       [   0,    0,    0, ..., 7648, 9166, 1564],
       [   0,    0,    0, ..., 1656, 5435, 9166]])

In [13]:
train_data.shape

(53, 39)

In [20]:
len(tag)

8

In [21]:
tag

['greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'name',
 'options',
 'south_africa_info',
 'south_africa_facts']

In [24]:
encode = {'greeting':0,'goodbye':1,'thanks':2,'noanswer':3,'name':4,'options':5,'south_africa_info':6,'south_africa_facts':7}
target = []
for label in labels:
    target.append(encode[label])


In [11]:
tag = ['ram','shyam','mohit','kunal']
labels = ['ram','shyam','mohit','kunal','ram','shyam','mohit','kunal','ram','shyam','mohit','kunal']
target = []
dictionary = {}
for ind , value in enumerate(tag):
    dictionary[value] = ind
for label in labels:
    target.append(dictionary[label])
dictionary 
target

[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]

In [17]:
# target

In [25]:
vocabulary_size,maximum_length_of_sentence

(10000, 39)

In [26]:
model  = Sequential()
model.add(Embedding(vocabulary_size,500,input_length=maximum_length_of_sentence))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [27]:
train_data.shape

(53, 39)

In [ ]:
label = np.array(t)

In [38]:
labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'options',
 'options',
 'options',
 'options',
 'options',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts']

In [39]:
pattern

['Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day',
 'Hey',
 'Ekse',
 'Hi',
 'Bye',
 'See you later',
 'Goodbye',
 'Ok bye',
 'Bye Bye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Awesome, thanks',
 'Thanks for helping me',
 'What is your name',
 'Whats your name',
 'Tell me your name',
 'Who are you',
 'Ungubani',
 'Tell me about yourself',
 'You are',
 'What do you do',
 'How can you help',
 'What do you know',
 'What is your purpose',
 'How can you help',
 'South Africa',
 'What can you tell me about SA',
 'SA',
 'Tell me about SA',
 'What about SA',
 'What do you know about SA',
 'Tell me more about SA',
 'What can you tell me about South Africa',
 'Tell me about South Africa',
 'What about South Africa',
 'What do you know about South Africa',
 'Tell me more about South Africa',
 'South Africa Facts',
 'SA Facts',
 'Give me some facts about SA',
 'Give me some facts about South Africa',
 'SA Facts',
 'SA interesting facts',
 'South Africa f

In [10]:
response

[['Hi stranger',
  'Yebo yes, how can I help',
  'Ey watsupp',
  'Ekse Hoezit',
  'Hola',
  'Hey dude',
  'You again'],
 ['See you!',
  'Have a nice day',
  'Sure Bye',
  'Later dude',
  'Sayoonara',
  'Peace out',
  'Sure dude',
  'Ciao'],
 ['Happy to help!',
  'Any time!',
  'My pleasure',
  'You are welcome',
  'Sure dude'],
 ["Sorry, can't understand you",
  'Please give me more info',
  'Not sure I understand',
  'I am still here dude'],
 ['I am Karabo',
  'My name is Karabo',
  'You can call me KB for short',
  'I am Karabo, my friends call me KB',
  'Karabo is my name'],
 ['I can tell you about South Africa',
  'I can give you interesting facts about South Africa'],
 ['South Africa is the southern most country in Africa, it has a population of more than 59million people and covers an area of 1221 037m2. South Africa has 11 official languages. The currency used in ZAR our South African Rand which trades at about 15ZAR per 1USD.'],
 ['South Africa is now the only country in the wo

In [9]:
tag

['greeting',
 'goodbye',
 'thanks',
 'noanswer',
 'name',
 'options',
 'south_africa_info',
 'south_africa_facts']

In [10]:
len(pattern),len(labels)

## independent and dependent variables

(53, 53)

In [5]:
tokenizer = Tokenizer(oov_token='<nothing>')

In [6]:
tokenizer.fit_on_texts(pattern)

In [7]:
word_index = tokenizer.word_index

In [8]:
sequence = tokenizer.texts_to_sequences(pattern)

In [9]:
max_sequnce_len = max(map(len,pattern))

In [10]:
sequences = pad_sequences(sequences=sequence ,maxlen=max_sequnce_len,padding='post')

In [11]:
sequences
## xtrain_data

array([[22, 23,  0, ...,  0,  0,  0],
       [15, 16,  2, ...,  0,  0,  0],
       [17, 29, 23, ...,  0,  0,  0],
       ...,
       [ 6,  7, 10, ...,  0,  0,  0],
       [ 6,  7, 21, ...,  0,  0,  0],
       [ 9,  4, 50, ...,  0,  0,  0]])

In [13]:
sequences.shape

(53, 39)

In [24]:
encode = {'greeting':0,'goodbye':1,'thanks':2,'name':3,'options':4,'south_africa_info':5,'south_africa_facts':6}
target = []
for label in labels:
    target.append(encode[label])

    

In [16]:
labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'options',
 'options',
 'options',
 'options',
 'options',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_info',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts',
 'south_africa_facts']

In [25]:
target = np.array(target)
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       6, 6, 6, 6, 6, 6, 6, 6, 6])

In [54]:
len(target)

53

In [55]:
# labels = pd.get_dummies(labels).values
# labels

In [14]:
vocab_size = len(word_index)
embedding_dim = 2
vocab_size

50

In [19]:
sequences.shape,labels.shape

((53, 39), (53, 7))

In [23]:
sequences[0].shape

(39,)

In [56]:
vocab_size

50

In [105]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [99]:
algo = RandomForestClassifier()

In [100]:
algo.fit(x_train,target)

RandomForestClassifier()

In [101]:
algo.score(x_train,target)

1.0

In [102]:
y_pred = algo.predict(x_train)

In [103]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       7, 7, 7, 7, 7, 7, 7, 7, 7])

In [104]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       7, 7, 7, 7, 7, 7, 7, 7, 7])

In [106]:
confusion_matrix(y_pred,target)

array([[10,  0,  0,  0,  0,  0,  0],
       [ 0,  5,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  0,  0,  0],
       [ 0,  0,  0,  7,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  0,  0],
       [ 0,  0,  0,  0,  0, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  9]], dtype=int64)

In [109]:
import pickle
os.makedirs('models',exist_ok=True)
pickle.dump(algo,open('models/model.pkl','wb'))
pickle.dump(response,open('models/response.pkl','wb'))

### now i can easily classify the intents


### and now i wants to get the most suitable respose from the corpus

In [110]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer,util
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import pandas as pd
import re
import os
import string
import pickle

In [111]:
pattern

['Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day',
 'Hey',
 'Ekse',
 'Hi',
 'Bye',
 'See you later',
 'Goodbye',
 'Ok bye',
 'Bye Bye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Awesome, thanks',
 'Thanks for helping me',
 'What is your name',
 'Whats your name',
 'Tell me your name',
 'Who are you',
 'Ungubani',
 'Tell me about yourself',
 'You are',
 'What do you do',
 'How can you help',
 'What do you know',
 'What is your purpose',
 'How can you help',
 'South Africa',
 'What can you tell me about SA',
 'SA',
 'Tell me about SA',
 'What about SA',
 'What do you know about SA',
 'Tell me more about SA',
 'What can you tell me about South Africa',
 'Tell me about South Africa',
 'What about South Africa',
 'What do you know about South Africa',
 'Tell me more about South Africa',
 'South Africa Facts',
 'SA Facts',
 'Give me some facts about SA',
 'Give me some facts about South Africa',
 'SA Facts',
 'SA interesting facts',
 'South Africa f

In [112]:
response


[['Hi stranger',
  'Yebo yes, how can I help',
  'Ey watsupp',
  'Ekse Hoezit',
  'Hola',
  'Hey dude',
  'You again'],
 ['See you!',
  'Have a nice day',
  'Sure Bye',
  'Later dude',
  'Sayoonara',
  'Peace out',
  'Sure dude',
  'Ciao'],
 ['Happy to help!',
  'Any time!',
  'My pleasure',
  'You are welcome',
  'Sure dude'],
 ["Sorry, can't understand you",
  'Please give me more info',
  'Not sure I understand',
  'I am still here dude'],
 ['I am Karabo',
  'My name is Karabo',
  'You can call me KB for short',
  'I am Karabo, my friends call me KB',
  'Karabo is my name'],
 ['I can tell you about South Africa',
  'I can give you interesting facts about South Africa'],
 ['South Africa is the southern most country in Africa, it has a population of more than 59million people and covers an area of 1221 037m2. South Africa has 11 official languages. The currency used in ZAR our South African Rand which trades at about 15ZAR per 1USD.'],
 ['South Africa is now the only country in the wo

In [ ]:
util.cos_sim(,embed_question)

In [20]:
vocab_size = len(word_index)

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

In [26]:
model  = Sequential()
model.add(Embedding(vocab_size,20,input_length=39))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(7,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 39, 20)            1000      
                                                                 
 dropout (Dropout)           (None, 39, 20)            0         
                                                                 
 lstm (LSTM)                 (None, 200)               176800    
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 8)                 1608      
                                                                 
Total params: 179,408
Trainable params: 179,408
Non-trainable params: 0
_________________________________________________________________


In [21]:
sequences.shape,target.shape

((53, 39), (53,))

In [22]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       7, 7, 7, 7, 7, 7, 7, 7, 7])

In [27]:
model.fit(sequences,target,epochs=1)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/embedding_2/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Ranjit Singh\AppData\Local\Prog

In [29]:
model = tf.keras.Sequential()
# model.add(Embedding(vocab_size, 20,input_length=39)
model.add(LSTM(39))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [23]:
sequences.shape,target.shape

((53, 39), (53,))

In [34]:
history = model.fit(sequences,target,epochs=1)

ValueError: in user code:

    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb)

In [64]:
def create_ann_model(input_shape, num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [72]:
encode

{'greeting': 1,
 'goodbye': 2,
 'thanks': 3,
 'name': 4,
 'options': 5,
 'south_africa_info': 6,
 'south_africa_facts': 7}

In [74]:
model = create_ann_model(x_train.shape,6)

In [75]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 53, 128)           5120      
                                                                 
 dropout_8 (Dropout)         (None, 53, 128)           0         
                                                                 
 dense_17 (Dense)            (None, 53, 64)            8256      
                                                                 
 dropout_9 (Dropout)         (None, 53, 64)            0         
                                                                 
 dense_18 (Dense)            (None, 53, 6)             390       
                                                                 
Total params: 13,766
Trainable params: 13,766
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.fit(x_train,target, epochs=1)

ValueError: in user code:

    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb)

In [58]:
x_train = np.array(sequences)

In [59]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       7, 7, 7, 7, 7, 7, 7, 7, 7])

In [60]:
x_train.shape,target.shape

((53, 39), (53,))

In [61]:
y_train

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0,

In [62]:
history = model.fit(x_train,target , epochs=1)

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_l

In [53]:
### model defining


import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, GlobalMaxPooling1D, Dense

def create_lstm_model(vocab_size, embedding_dim, lstm_units, classes):
    model = tf.keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim,input_length=39))
    model.add(LSTM(lstm_units))
#     model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(classes, activation='softmax'))

    return model

In [54]:
model = create_lstm_model(vocab_size,embedding_dim,112,7)

In [55]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 39, 2)             100       
                                                                 
 lstm_4 (LSTM)               (None, 112)               51520     
                                                                 
 dense_4 (Dense)             (None, 64)                7232      
                                                                 
 dense_5 (Dense)             (None, 7)                 455       
                                                                 
Total params: 59,307
Trainable params: 59,307
Non-trainable params: 0
_________________________________________________________________


In [57]:
history = model.fit(sequences,labels , epochs=1)


1/2 [==============>...............] - ETA: 2s - loss: 1.9451 - accuracy: 0.3125

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_6/embedding_4/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\Ranjit Singh\AppData\Local\Prog

In [45]:
sequences

array([[22, 23,  0, ...,  0,  0,  0],
       [15, 16,  2, ...,  0,  0,  0],
       [17, 29, 23, ...,  0,  0,  0],
       ...,
       [ 6,  7, 10, ...,  0,  0,  0],
       [ 6,  7, 21, ...,  0,  0,  0],
       [ 9,  4, 50, ...,  0,  0,  0]])

In [46]:
labels

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0,

4